## Get the Nodes around Tolworth Test centre

In [1]:
# Import libraries
import osmnx as ox
import networkx as nx
import dash_leaflet as dl
from jupyter_dash import JupyterDash
from dash import html
import dash

In [2]:
# Get the Graph and the Node
G = ox.graph_from_point((51.3829463, -0.2933327), dist=4000, network_type='drive')  # Adjust lat/lon to roughly London
node_id = 10282513496

# if node_id not in G:
#     raise ValueError("Node not found in graph")

# centre_node = G.nodes[node_id]
# centre_point = (centre_node['y'], centre_node['x'])

# centre_node
#G.edges

G.add_node('test_centre', x=-0.29299, y=51.38413)
G.add_node('test_centre_road', x=-0.29280, y=51.38409)

In [3]:
G.nodes(data=True)#['test_centre_road'], G.nodes[304064]

NodeDataView({292217: {'y': 51.3756472, 'x': -0.3074197, 'street_count': 3}, 292250: {'y': 51.3740534, 'x': -0.297643, 'street_count': 3}, 304051: {'y': 51.375014, 'x': -0.3042848, 'street_count': 3}, 304060: {'y': 51.3753315, 'x': -0.3040909, 'street_count': 3}, 304061: {'y': 51.375324, 'x': -0.3038206, 'street_count': 3}, 304064: {'y': 51.3785644, 'x': -0.3024115, 'street_count': 4}, 304076: {'y': 51.3828974, 'x': -0.3005723, 'street_count': 3}, 304081: {'y': 51.3866926, 'x': -0.3008888, 'street_count': 3}, 304434: {'y': 51.385394, 'x': -0.2955969, 'street_count': 4}, 304435: {'y': 51.3849962, 'x': -0.2945233, 'street_count': 3}, 304436: {'y': 51.3847833, 'x': -0.2938141, 'street_count': 3}, 304437: {'y': 51.3844458, 'x': -0.2926802, 'street_count': 3}, 304438: {'y': 51.383925, 'x': -0.2909422, 'street_count': 3}, 304440: {'y': 51.38292, 'x': -0.2875149, 'street_count': 3}, 304441: {'y': 51.382604, 'x': -0.2862621, 'street_count': 3}, 304455: {'y': 51.3854974, 'x': -0.2807027, 'stree

In [3]:
centre_lat = 51.3829463
centre_lon = -0.2933327

# Get nearby nodes
nearby_nodes = [
    (nid, data) for nid, data in G.nodes(data=True)
    #if ox.distance.great_circle(centre_lat, centre_lon, data['y'], data['x']) <= 1000
]

markers = [
    dl.Marker(position=(data['y'], data['x']), children=dl.Tooltip(f'ID: {nid}, x: {data['x']}, y: {data['y']}'))
    for nid, data in nearby_nodes
]

#dx = -2:-1
nearby_nodes[-3:-1], markers[-3:-1]


([(12879656677, {'y': 51.4157366, 'x': -0.2845178, 'street_count': 2}),
  ('test_centre', {'x': -0.29299, 'y': 51.38413})],
 [Marker(children=Tooltip('ID: 12879656677, x: -0.2845178, y: 51.4157366'), position=(51.4157366, -0.2845178)),
  Marker(children=Tooltip('ID: test_centre, x: -0.29299, y: 51.38413'), position=(51.38413, -0.29299))])

In [ ]:
for nid, data in G.nodes(data=True):
    print(nid, data)

In [4]:
app = JupyterDash(__name__)
app.layout = html.Div([
    dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
        dl.TileLayer(),
        dl.LayerGroup(markers)
    ], style={'width': '100%', 'height': '4200px'})
])

app.run(mode='inline')

/home/dave/anaconda3/envs/mapping_py12/lib/python3.12/site-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [21]:
G.nodes[1370329122], G.get_edge_data(745985600, 1370329122)

({'y': 51.3843892,
  'x': -0.319595,
  'highway': 'mini_roundabout',
  'street_count': 3},
 {0: {'osmid': 122698546,
   'highway': 'tertiary',
   'name': 'Fleece Road',
   'oneway': False,
   'reversed': True,
   'length': np.float64(13.050001099634002),
   'geometry': <LINESTRING (-0.319 51.384, -0.32 51.384, -0.32 51.384)>}})

In [7]:
# Example node IDs
# u = 134929439  # source node
# v = 9883509218  # target node

# # Get list of edges between u and v (could be multiple if multigraph)
# edges = G.get_edge_data(u, v)  # returns dict of edge keys -> edge data

# # Example: print info
# for key, edge_data in edges.items():
#     print(f"Edge key: {key}")
#     print(f"Length: {edge_data['length']} meters")


In [8]:
node_data = G.nodes[9883509218]
print(node_data)

KeyError: 9883509218

In [ ]:
from shapely.geometry import LineString

def edge_to_coords(edge_data, u, v):
    if 'geometry' in edge_data:
        return [(point[1], point[0]) for point in edge_data['geometry'].coords]
    else:
        # fallback: straight line between nodes
        return [
            (G.nodes[u]['y'], G.nodes[u]['x']),
            (G.nodes[v]['y'], G.nodes[v]['x'])
        ]

# Collect polylines
edge_lines = [edge_to_coords(data, u, v) for key, data in edges.items()]

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
def get_edge_name(edge_data):
    name = edge_data.get("name", "Unnamed road")
    if isinstance(name, list):
        return ", ".join(name)
    return name

edge_polylines = [
    dl.Polyline(
        positions=edge_to_coords(data, u, v),
        color='red',
        weight=5,
        children=dl.Tooltip(get_edge_name(data))
    )
    for key, data in edges.items()
]


In [ ]:
# mapp = dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
#     dl.TileLayer(),
#     dl.LayerGroup([
#         *markers,  # existing markers from previous steps
#         *edge_polylines
#     ], style={'width': '100%', 'height': '600px'})
# ])

# app = JupyterDash(__name__)
# app.layout = html.Div([
#     dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
#     dl.TileLayer(),
#     dl.LayerGroup([
#         markers,  # existing markers from previous steps
#         edge_polylines
#     ])
# ])
# ])

# app.run(mode='inline')

app = JupyterDash(__name__)
app.layout = html.Div([
    dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
        dl.TileLayer(),
        dl.LayerGroup(markers),
        dl.LayerGroup(edge_polylines)
    ], style={'width': '100%', 'height': '600px'})
])

app.run(mode='inline')

/home/dave/anaconda3/envs/mapping_py12/lib/python3.12/site-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [ ]:
def get_nodes_on_street(G, street_name):
    nodes = set()
    for u, v, data in G.edges(data=True):
        name = data.get("name")
        if isinstance(name, list):
            if street_name in name:
                nodes.update([u, v])
        elif name == street_name:
            nodes.update([u, v])
    return list(nodes)

# Get nodes on Douglas road
Douglas_nodes = get_nodes_on_street(G, 'Douglas Road')

# Get nodes on Dennan road
DennanRd_nodes = get_nodes_on_street(G, 'Dennan Road')



Douglas_nodes, DennanRd_nodes

([9883509218,
  1962249571,
  23779844,
  9883509220,
  23780711,
  2578590287,
  134912281,
  134912282,
  134929439],
 [23787122, 23780562, 23779844, 23780702])